# 参数初始化

**本章内容作为补充内容，可以后期有需要使用别人已经预训练好了的模型再回来翻**


对于PyTorch中的各种网络层，我们会需要对参数进行初始化的操作，这里我们以 nn.Linear() 为例子：

```
class Linear(Module):
    def __init__(self, in_features, out_features, bias=True):
    ...
    ...
    ...
        self.reset_parameters()
    ...
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
```

上述代码摘自 PyTorch [torch.nn.Linear()的源码](http://pytorch.org/docs/master/_modules/torch/nn/modules/linear.html#Linear)

从源代码中可以看到，PyTorch在我们实例化某层的时候，就帮我们自动完成了初始化的操作。但是，有时候，我们可能需要一些不一样的初始化。

比如使用 xavier 方法 来进行初始化，我们该怎么做呢？

在PyTorch中有一个 torch.nn.init 这个包就是用来帮我们完成初始化的。

这个包提供了很多初始化的方法：

* torch.nn.init.constant(tensor, val)
* torch.nn.init.normal(tensor, mean=0, std=1)
* torch.nn.init.xavier_uniform(tensor, gain=1)

详情 http://pytorch.org/docs/master/nn.html#torch-nn-init


**注意： PyTorch 中各个层默认的初始化是 uniform —— 均匀分布初始化**


In [1]:
import torch
import torch.nn as nn

class LeNet5(nn.Module):
    def __init__(self, in_dim, n_class):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, n_class)
        
        # 参数初始化函数
        for p in self.modules():
            if isinstance(p, nn.Conv2d):
                nn.init.xavier_normal(p.weight.data)
            elif isinstance(p, nn.Linear):
                nn.init.normal(p.weight.data)

    # 向前传播
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


# 对层进行处理

现在，我们得到了一个 LeNet，实例化之后打印，结果如下所示：

```
LeNet5(
  (conv1): Conv2d(224, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
```

然后，我们现在需要把所有的带有 "conv" 的层全部删除（替换）我们该怎么做呢？

我们需要重新写一个模型嘛？

NOOOOO！！！

不需要， Pytorch 的层是可以随意替换的！！！我们随时可以增加，修改。

In [2]:
import re

# 实例化 LeNet

lenet = LeNet5(224, 10)

del_list = [] # 将所有需要删除的层的名字放进这列表里面
for name, module in lenet.named_children():
    # 对所有的层的名字进行匹配，如果层中含有conv则把名字放入等待删除列表中
    if re.match("conv", name) != None:
        del_list.append(name)

# 获取了所有带 "conv" 层的名字了，现在开始删除
for name in del_list:
    delattr(lenet, name)

print(lenet) 得到的结果如下所示

```
LeNet5(
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
```

# 使用预训练的AlexNet模型来对CIFAR10进行训练

现在，我们已经知道AlexNet是个好用的模型，我们想用它来对CIFAR10进行训练，但是我们不想自己训练，想用别人训练好的，我们该怎么做呢？

分三步：

1、加载预训练模型

2、对模型结构进行更改

3、重新训练，我们需要进行训练的那几层

In [3]:
from torchvision import models, transforms, datasets
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.optim as optim

In [4]:
# -----------------get Alexnet model-------------------------
def getAlexNet(DOWNLOAD=True):
    alexnet = models.alexnet(pretrained=DOWNLOAD)
    return alexnet


# 把 tensor 变成 Variable
def to_var(x):
    x = Variable(x)
    if torch.cuda.is_available():
        x = x.cuda()
    return x

# 是否下载
DOWNLOAD = True

# 需要分类的类别
CLASS_NUM = 10

# 下载预训练好的AlexNet模型
alexnet = getAlexNet(DOWNLOAD)


In [5]:
# -----------------修改预训练好的 Alexnet 模型中的分类层-------------------------
alexnet.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(in_features=9216, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=4096, out_features=CLASS_NUM, bias=True)
)

In [6]:
# 使用GPU
if torch.cuda.is_available():
    alexnet = alexnet.cuda()

# 数据预处理
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 加载数据
train_data = datasets.CIFAR10('./data', train=True, transform=transform, download=True)
test_data = datasets.CIFAR10('./data', train=False, transform=transform, download=False)

train_data_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=256, shuffle=True)

# optim
learning_rate = 0.0001
num_epoches = 5
criterion = nn.CrossEntropyLoss()

# 训练的时候，我们只更新 classifier 层的参数
optimizer = optim.Adam(alexnet.classifier.parameters(), lr=learning_rate)

# training
alexnet.train()
for epoch in range(num_epoches):
    print('epoch {}'.format(epoch + 1))
    runnin_acc = 0.0
    running_loss = 0.0
    for data, label in train_data_loader:
        img = to_var(data)
        label = to_var(label)
        out = alexnet(img)
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        accuracy = (pred == label).float().mean()
        runnin_acc += num_correct.data[0]
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Acc: {:.6f}".format(runnin_acc / len(train_data)))

# evaluation
alexnet.eval()
runnin_acc = 0.0
for data, label in test_data_loader:
    img = to_var(data)
    label = to_var(label)
    out = alexnet(img)
    loss = criterion(out, label)
    _, pred = torch.max(out, 1)
    num_correct = (pred == label).sum()
    accuracy = (pred == label).float().mean()
    runnin_acc += num_correct.data[0]
print("Acc: {:.6f}".format(runnin_acc / len(test_data)))


print("Done!")


Files already downloaded and verified
epoch 1
Acc: 0.740700
epoch 2
Acc: 0.821440
epoch 3
Acc: 0.842560
epoch 4
Acc: 0.863100
epoch 5
Acc: 0.877080
Acc: 0.857800
Done!
